# Categorical preprocessors

> Covering use cases not handled by native scikit-learn transformers.

In [ ]:
# | default_exp preprocessing.categorical

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export


from __future__ import annotations
import collections
from typing import Union, List

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from sklearn.utils.fixes import _object_dtype_isnan
from sklearn.utils.validation import check_is_fitted

from poniard.utils.estimate import get_target_info

In [ ]:
# | exporti
# | hide


def check_input(X):
    X_ = check_array(X, dtype=None, ensure_2d=True, force_all_finite=False)
    # If the array contains both NaNs and strings, convert to object type
    if X_.dtype.kind in {"U", "S"}:  # contains strings
        if np.any(X_ == "nan"):  # missing value converted to string
            return check_array(
                np.array(X, dtype=object),
                dtype=None,
                ensure_2d=True,
                force_all_finite=False,
            )
    return X_

In [ ]:
# | export


class TargetEncoder(BaseEstimator, TransformerMixin):
    """Encode categorical features considering the effect that it has in the
    target variable.

    Note that implementation and docstrings are largely taken from
    [Dirty Cat](https://github.com/dirty-cat/dirty_cat/blob/master/dirty_cat/target_encoder.py).

    Parameters
    ----------
    task :
        The type of problem. Either "classification" or "regression".
    handle_unknown :
        Either "error" or "ignore". Whether to raise an error or ignore if a unknown
        categorical feature is present during transform (default is to raise). If 'ignore',
        unknown categories will be set to the mean of the target.
    handle_missing :
        Either "error" or "". Whether to raise an error or impute with blank string "" if missing
        values (NaN) are present during fit (default is to impute).
    """

    def __init__(self, task: str, handle_unknown="error", handle_missing=""):
        self.task = task
        self.handle_unknown = handle_unknown
        self.handle_missing = handle_missing

    def _more_tags(self):
        """
        Used internally by sklearn to ease the estimator checks.
        """
        return {"X_types": ["categorical"]}

    def fit(
        self,
        X: Union[pd.DataFrame, np.ndarray, List],
        y: Union[pd.DataFrame, np.ndarray, List],
    ) -> TargetEncoder:
        """Fit the TargetEncoder to X.

        Parameters
        ----------
        X :
            The data to determine the categories of each feature.
        y :
            The associated target vector.

        Returns
        -------
        TargetEncoder
            Fitted TargetEncoder.
        """
        self.type_of_target_ = get_target_info(y, self.task)["type_"]
        if isinstance(X, pd.DataFrame):
            self.colnames_ = X.columns
        X = check_input(X)
        self.n_features_in_ = X.shape[1]
        X = X.astype(str)
        if self.handle_missing not in ["error", ""]:
            template = "handle_missing should be either 'error' or " "'', got %s"
            raise ValueError(template % self.handle_missing)
        if hasattr(X, "iloc") and X.isna().values.any():
            if self.handle_missing == "error":
                msg = (
                    "Found missing values in input data; set "
                    "handle_missing='' to encode with missing values"
                )
                raise ValueError(msg)
            else:
                X = X.fillna(self.handle_missing)
        elif not hasattr(X, "dtype") and isinstance(X, list):
            X = np.asarray(X, dtype=object)
        if hasattr(X, "dtype"):
            mask = _object_dtype_isnan(X)
            if mask.any():
                if self.handle_missing == "error":
                    msg = (
                        "Found missing values in input data; set "
                        "handle_missing='' to encode with missing values"
                    )
                    raise ValueError(msg)
                else:
                    X[mask] = self.handle_missing

        if self.handle_unknown not in ["error", "ignore"]:
            template = "handle_unknown should be either 'error' or " "'ignore', got %s"
            raise ValueError(template % self.handle_unknown)
        X_temp = check_array(X, dtype=None)
        if not hasattr(X, "dtype") and np.issubdtype(X_temp.dtype, np.str_):
            X = check_array(X, dtype=np.object)
        else:
            X = X_temp

        n_features = X.shape[1]

        self._label_encoders_ = [LabelEncoder() for _ in range(n_features)]

        for j in range(n_features):
            le = self._label_encoders_[j]
            Xj = X[:, j]
            le.fit(Xj)
        self.categories_ = [le.classes_ for le in self._label_encoders_]
        self.n_ = len(y)
        if self.type_of_target_ == "binary" and np.array(y).dtype.kind not in "biufc":
            y = LabelEncoder().fit_transform(y)
        if self.type_of_target_ in ["continuous", "binary"]:
            self.Eyx_ = [
                {cat: np.mean(y[X[:, j] == cat]) for cat in self.categories_[j]}
                for j in range(len(self.categories_))
            ]
            self.Ey_ = np.mean(y)
            self.counter_ = {j: collections.Counter(X[:, j]) for j in range(n_features)}
        if self.type_of_target_ == "multiclass":
            self.classes_ = np.unique(y)

            self.Eyx_ = {
                c: [
                    {
                        cat: np.mean((y == c)[X[:, j] == cat])
                        for cat in self.categories_[j]
                    }
                    for j in range(len(self.categories_))
                ]
                for c in self.classes_
            }
            self.Ey_ = {c: np.mean(y == c) for c in self.classes_}
            self.counter_ = {j: collections.Counter(X[:, j]) for j in range(n_features)}
        self.k_ = {j: len(self.counter_[j]) for j in self.counter_}
        return self

    def transform(self, X: Union[pd.DataFrame, np.ndarray, List]):
        """Transform X using specified encoding scheme.

        Parameters
        ----------
        X :
            The data to encode.
        Returns
        -------
        X :
            Transformed input.
        """
        check_is_fitted(self, attributes=["n_features_in_"])
        X = check_input(X)
        if X.shape[1] != self.n_features_in_:
            raise ValueError(
                f"Number of features in the input data ({X.shape[1]}) does not match the number of features "
                f"seen during fit ({self.n_features_in_})."
            )
        X = X.astype(str)
        if hasattr(X, "iloc") and X.isna().values.any():
            if self.handle_missing == "error":
                msg = (
                    "Found missing values in input data; set "
                    "handle_missing='' to encode with missing values"
                )
                raise ValueError(msg)
            else:
                X = X.fillna(self.handle_missing)
        elif not hasattr(X, "dtype") and isinstance(X, list):
            X = np.asarray(X, dtype=object)
        if hasattr(X, "dtype"):
            mask = _object_dtype_isnan(X)
            if mask.any():
                if self.handle_missing == "error":
                    msg = (
                        "Found missing values in input data; set "
                        "handle_missing='' to encode with missing values"
                    )
                    raise ValueError(msg)
                else:
                    X[mask] = self.handle_missing

        X_temp = check_array(X, dtype=None)
        if not hasattr(X, "dtype") and np.issubdtype(X_temp.dtype, np.str_):
            X = check_array(X, dtype=np.object)
        else:
            X = X_temp

        n_features = X.shape[1]
        X_int = np.zeros_like(X, dtype=int)
        X_mask = np.ones_like(X, dtype=bool)

        for i in range(n_features):
            Xi = X[:, i]
            valid_mask = np.in1d(Xi, self.categories_[i])

            if not np.all(valid_mask):
                if self.handle_unknown == "error":
                    diff = np.unique(X[~valid_mask, i])
                    msg = (
                        "Found unknown categories {0} in column {1}"
                        " during transform".format(diff, i)
                    )
                    raise ValueError(msg)
                else:
                    # Set the problematic rows to an acceptable value and
                    # continue `The rows are marked `X_mask` and will be
                    # removed later.
                    X_mask[:, i] = valid_mask
                    Xi = Xi.copy()
                    Xi[~valid_mask] = self.categories_[i][0]
            X_int[:, i] = self._label_encoders_[i].transform(Xi)

        out = []

        for j, cats in enumerate(self.categories_):
            unqX = np.unique(X[:, j])
            encoder = {x: 0 for x in unqX}
            if self.type_of_target_ in ["continuous", "binary"]:
                for x in unqX:
                    if x not in cats:
                        Eyx = 0
                    else:
                        Eyx = self.Eyx_[j][x]
                    lambda_n = self.lambda_(self.counter_[j][x], self.n_ / self.k_[j])
                    encoder[x] = lambda_n * Eyx + (1 - lambda_n) * self.Ey_
                x_out = np.zeros((len(X[:, j]), 1))
                for i, x in enumerate(X[:, j]):
                    x_out[i, 0] = encoder[x]
                out.append(x_out.reshape(-1, 1))
            if self.type_of_target_ == "multiclass":
                x_out = np.zeros((len(X[:, j]), len(self.classes_)))
                lambda_n = {x: 0 for x in unqX}
                for x in unqX:
                    lambda_n[x] = self.lambda_(
                        self.counter_[j][x], self.n_ / self.k_[j]
                    )
                for k, c in enumerate(np.unique(self.classes_)):
                    for x in unqX:
                        if x not in cats:
                            Eyx = 0
                        else:
                            Eyx = self.Eyx_[c][j][x]
                        encoder[x] = lambda_n[x] * Eyx + (1 - lambda_n[x]) * self.Ey_[c]
                    for i, x in enumerate(X[:, j]):
                        x_out[i, k] = encoder[x]
                out.append(x_out)
        out = np.hstack(out)
        return out

    def get_feature_names_out(self, input_features=None) -> List[str]:
        """Get feature names for output."""
        colnames_ = getattr(self, "colnames_", None)
        if colnames_ is not None:
            if self.type_of_target_ in ["continuous", "binary"]:
                output = colnames_.tolist()
            else:
                output = []
                for name in colnames_:
                    unique = self.classes_
                    names = [f"{name}_{level}" for level in unique]
                    output.extend(names)
            return output

    def get_feature_names(self, input_features=None) -> List[str]:
        return self.get_feature_names_out()

    @staticmethod
    def lambda_(x, n):
        out = x / (x + n)
        return out

In general, `TargetEncoder` takes the ratio between the mean of the target for a given category and the mean of the target. In addition, it takes an empirical Bayes approach to shrink the estimate.

It is particularly useful with high cardinality categoricals, as it will not expand the feature space as much as one hot encoding, but retains more information than ordinal encoding.

For more details, see [Micci-Barreca, 2001: A preprocessing scheme for high-cardinality categorical attributes in classification and prediction problems](https://dl.acm.org/doi/10.1145/507533.507538).

In [ ]:
show_doc(TargetEncoder.fit)

---

[source](https://github.com/rxavier/poniard/blob/master/poniard/preprocessing/categorical.py#L67){target="_blank" style="float:right; font-size:smaller"}

### TargetEncoder.fit

>      TargetEncoder.fit
>                         (X:Union[pandas.core.frame.DataFrame,numpy.ndarray,Lis
>                         t], y:Union[pandas.core.frame.DataFrame,numpy.ndarray,
>                         List])

Fit the TargetEncoder to X.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| X | Union[pd.DataFrame, np.ndarray, List] | The data to determine the categories of each feature. |
| y | Union[pd.DataFrame, np.ndarray, List] | The associated target vector. |
| **Returns** | **TargetEncoder** | **Fitted TargetEncoder.** |

After fitting, the categories of each feature are held in the `categories_` attribute.

In [ ]:
show_doc(TargetEncoder.transform)

---

[source](https://github.com/rxavier/poniard/blob/master/poniard/preprocessing/categorical.py#L164){target="_blank" style="float:right; font-size:smaller"}

### TargetEncoder.transform

>      TargetEncoder.transform
>                               (X:Union[pandas.core.frame.DataFrame,numpy.ndarr
>                               ay,List])

Transform X using specified encoding scheme.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| X | Union[pd.DataFrame, np.ndarray, List] | Transformed input. |

In [ ]:
show_doc(TargetEncoder.fit_transform)

---

### TransformerMixin.fit_transform

>      TransformerMixin.fit_transform (X, y=None, **fit_params)

Fit to data, then transform it.

Fits transformer to `X` and `y` with optional parameters `fit_params`
and returns a transformed version of `X`.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| X | array-like of shape (n_samples, n_features) |  | Input samples. |
| y | NoneType | None | Target values (None for unsupervised transformations). |
| fit_params |  |  |  |
| **Returns** | **ndarray array of shape (n_samples, n_features_new)** |  | **Transformed array.** |

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
rng = np.random.default_rng(0)

X = pd.DataFrame(
    {
        "sex": rng.choice(["female", "male", "other"], size=10),
        "status": rng.choice(
            ["employed", "unemployed", "retired", "inactive"], size=10
        ),
    }
)
y = rng.choice(["low", "high"], size=10)

encoder = TargetEncoder(task="classification", handle_unknown="ignore")
pd.DataFrame(encoder.fit_transform(X, y), columns=encoder.get_feature_names_out())

,sex,status
0,0.437500,0.416667
1,0.250000,0.375000
2,0.250000,0.416667
3,0.464286,0.416667
4,0.464286,0.375000
5,0.464286,0.416667
6,0.464286,0.416667
7,0.464286,0.416667
8,0.464286,0.416667
9,0.437500,0.375000


In the case of a multiclass target, the encodings are computed separately for each label,
meaning that each feature will be expanded to as many unique levels in the target.

In [ ]:
y = rng.choice(["low", "mid", "high"], size=10)

encoder = TargetEncoder(task="classification", handle_unknown="ignore")
pd.DataFrame(encoder.fit_transform(X, y), columns=encoder.get_feature_names_out())

,sex_high,sex_low,sex_mid,status_high,status_low,status_mid
0,0.312500,0.3125,0.375000,0.250,0.458333,0.291667
1,0.125000,0.6875,0.187500,0.125,0.562500,0.312500
2,0.125000,0.6875,0.187500,0.250,0.458333,0.291667
3,0.178571,0.5000,0.321429,0.250,0.458333,0.291667
4,0.178571,0.5000,0.321429,0.125,0.562500,0.312500
5,0.178571,0.5000,0.321429,0.250,0.458333,0.291667
6,0.178571,0.5000,0.321429,0.250,0.458333,0.291667
7,0.178571,0.5000,0.321429,0.250,0.458333,0.291667
8,0.178571,0.5000,0.321429,0.250,0.458333,0.291667
9,0.312500,0.3125,0.375000,0.125,0.562500,0.312500


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()